🧪 Task 1 — NumPy + Pandas (Core Foundations)
🎯 Scenario

You are given raw sensor data collected every hour from a machine.
The data contains missing values, outliers, and needs basic analytics.

In [1]:
import numpy as np
import pandas as pd

data = {
    "hour": np.arange(1, 13),
    "temperature": [30, 32, None, 35, 100, 33, None, 34, 36, 37, None, 38],
    "pressure": [101, 102, 103, None, 500, 104, 105, None, 106, 107, 108, None]
}

df = pd.DataFrame(data)
print(df)


    hour  temperature  pressure
0      1         30.0     101.0
1      2         32.0     102.0
2      3          NaN     103.0
3      4         35.0       NaN
4      5        100.0     500.0
5      6         33.0     104.0
6      7          NaN     105.0
7      8         34.0       NaN
8      9         36.0     106.0
9     10         37.0     107.0
10    11          NaN     108.0
11    12         38.0       NaN


Task 1.1 — Data Cleaning

Replace None values in temperature and pressure with the mean of their respective columns.

Identify and remove outliers

Temperature > 50

Pressure > 200

In [12]:
df['temperature'] = df['temperature'].fillna(df['temperature'].mean()).round(1)
df['pressure'] = df['pressure'].fillna(df['pressure'].mean()).round(1)

df = df[(df['temperature'] <= 50) & (df['pressure'] <= 200)]
df
         

,hour,temperature,pressure
0,1,30.0,101.0
1,2,32.0,102.0
2,3,41.7,103.0
3,4,35.0,148.4
5,6,33.0,104.0
6,7,41.7,105.0
7,8,34.0,148.4
8,9,36.0,106.0
9,10,37.0,107.0
10,11,41.7,108.0


Task 1.2 — NumPy Operations

Convert cleaned temperature column into a NumPy array.

Normalize the temperature values using Min-Max scaling:

Task 1.3 — Pandas Analytics

Add a new column temp_category

"Low" → temp < 33

"Medium" → 33 ≤ temp ≤ 36

"High" → temp > 36

Task 1.4 — Final Output

Print:

Final cleaned DataFrame

Mean temperature per temp_category

In [24]:

df['temp_norm'] = df['temperature'].pipe(lambda x: x - x.min() / (x.max() - x.min())) 

conditions = [
    df['temp_norm'] < 33,
    df['temp_norm'].between(33, 36),
    df['temp_norm'] > 36
]
choices = ['Low', 'Medium', 'High']

df['temp_category'] = np.select(conditions,choices,default='Unknown')
df.groupby('temp_category')['temperature'].mean()

C:\Users\nilan\AppData\Local\Temp\ipykernel_30400\3772126980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_norm'] = df['temperature'].pipe(lambda x: x - x.min() / (x.max() - x.min()))
C:\Users\nilan\AppData\Local\Temp\ipykernel_30400\3772126980.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp_category'] = np.select(conditions,choices,default='Unknown')


temp_category
High      41.7
Low       32.8
Medium    37.0
Name: temperature, dtype: float64

🧪 Task 2 — Pandas + NumPy (Intermediate / Interview-Trap Level)

This task focuses on boolean masking, vectorization vs apply, and silent bugs — exactly what interviewers test for 3–4 years experience.

🎯 Scenario

You are analyzing employee performance data for a company.
The dataset has inconsistent scores, missing values, and requires derived metrics.

data = {
    "emp_id": [101, 102, 103, 104, 105, 106],
    "experience_years": [2, 5, None, 7, 3, None],
    "performance_score": [78, 88, 45, None, 92, 60],
    "department": ["IT", "HR", "IT", "Finance", "IT", "HR"]
}

🛠️ Tasks to Perform
Task 2.1 — Data Cleaning (Boolean Masking)

Fill missing experience_years with median.

Fill missing performance_score with mean.

Remove employees where performance_score < 50.

⚠️ Constraint:

Use boolean masking, not dropna() blindly.

Task 2.2 — Derived Column (Vectorized Logic)

Create a column rating:

"Excellent" → score ≥ 85

"Good" → 70 ≤ score < 85

"Average" → 50 ≤ score < 70

❌ No apply()
✅ Use np.select

Task 2.3 — NumPy-Based Analytics

Compute normalized performance score (Min–Max).

Compute a new column experience_weighted_score:

\text{weighted} = \text{performance_score} \times \log(1 + \text{experience_years})
weighted = performance_score 

⚠️ Use NumPy functions, not math.

Task 2.4 — Pandas Analytics

Find:

Average score per department

Best employee (highest weighted score) per department


In [60]:
import numpy as np
import pandas as pd

employee_data = {
    "emp_id": [101, 102, 103, 104, 105, 106],
    "experience_years": [2, 5, None, 7, 3, None],
    "performance_score": [78, 88, 45, None, 92, 60],
    "department": ["IT", "HR", "IT", "Finance", "IT", "HR"]
}

emp_df = pd.DataFrame(employee_data)

# Fill missing values
emp_df['experience_years'] = emp_df['experience_years'].fillna(
    emp_df['experience_years'].median()
)
emp_df['performance_score'] = emp_df['performance_score'].fillna(
    emp_df['performance_score'].mean()
)

# Remove low performers
emp_df = emp_df[emp_df['performance_score'] >= 50]

# Rating
conditions = [
    emp_df['performance_score'] >= 85,
    emp_df['performance_score'].between(70, 85, inclusive='left'),
    emp_df['performance_score'].between(50, 70, inclusive='left')
]
choices = ['Excellent', 'Good', 'Average']

emp_df['rating'] = np.select(conditions, choices, default='Unrated')

# Normalization
score = emp_df['performance_score']
emp_df['score_norm'] = (score - score.min()) / (score.max() - score.min())

# Weighted score
emp_df['weighted_score'] = (
    emp_df['performance_score'] *
    np.log1p(emp_df['experience_years'])
)

# Analytics
avg_score_dept = emp_df.groupby('department')['performance_score'].mean()

best_emp_dept = emp_df.loc[
    emp_df.groupby('department')['weighted_score'].idxmax(),
    ['department', 'emp_id', 'weighted_score']
] 


best_emp_dept

,department,emp_id,weighted_score
3,Finance,104,150.967456
1,HR,102,157.674833
4,IT,105,127.539081


🧪 Task 3 — Advanced Pandas & NumPy (Index Alignment + transform)
🎯 Scenario

You are working on sales performance analytics.
Each row is a sales transaction.
You need to compute relative performance metrics per region.

This task tests:

Index alignment awareness

groupby().transform() vs apply()

Vectorized multi-column logic

Subtle bugs that don’t throw errors

🛠️ Tasks to Perform
Task 3.1 — Data Cleaning

Fill missing sales_amount with region-wise mean
⚠️ Use groupby().transform(), NOT apply()

Task 3.2 — Performance Metrics

Create a column achievement_ratio
achievement_ratio=
target_amount
sales_amount
	​


Create a column performance_flag

"Exceeded" → ratio ≥ 1

"Met" → 0.9 ≤ ratio < 1

"Missed" → ratio < 0.9

❌ No loops
❌ No apply
✅ Use np.select

Task 3.3 — Relative Performance (IMPORTANT)

Create a column relative_to_region_avg
relative=sales_amount−(region average sales)

⚠️ This is where most people fail
You MUST:

Preserve row count

Preserve index alignment

Avoid merging

Task 3.4 — Analytics

For each region, find:

Total sales

Best sale (highest achievement_ratio)

In [ ]:
import numpy as np
import pandas as pd

data = {
    "sales_id": [1, 2, 3, 4, 5, 6, 7],
    "region": ["North", "South", "North", "West", "South", "North", "West"],
    "sales_amount": [1000, 1500, 1200, 800, None, 1100, 900],
    "target_amount": [900, 1400, 1000, 1000, 1300, 1000, 950]
}

df = pd.DataFrame(data)

sales_amount = df['sales_amount']
target_amount = df['target_amount']
sales_amount = sales_amount.fillna(df.groupby('region')['sales_amount'].transform('mean'))
df['sales_amount'] = sales_amount
achievement_ratio = sales_amount / target_amount
df['achievement_ratio'] = achievement_ratio.round(2)
performance_condt = [
    achievement_ratio >= 1,
    achievement_ratio.between(0.9,1,inclusive='both'),
    achievement_ratio < 0.9
]
performance_choices = [
    'Exceeded',
    'Met',
    'Missed'
]

df['performance_flag'] = np.select(performance_condt,performance_choices,default="NA")

region_avg_sales = df.groupby('region')['sales_amount'].transform('mean')
relative_performance = sales_amount -  region_avg_sales
print(region_avg_sales)
df['relative_performance'] = relative_performance

total_sale = df.groupby('region')['sales_amount'].sum()
best_sale = df.groupby('region')['achievement_ratio'].max()

pd.DataFrame([total_sale,best_sale])



0    1100.0
1    1500.0
2    1100.0
3     850.0
4    1500.0
5    1100.0
6     850.0
Name: sales_amount, dtype: float64


<bound method DataFrame.reset_index of         Total Sale  Achievement Ratio
region                               
North       3300.0               1.20
South       3000.0               1.15
West        1700.0               0.95>